In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv


load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

client = OpenAI(
    api_key = OPENAI_API_KEY
)

completion = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{'role':'user', 'content':'hi'}],
    temperature=0
)

print(completion.choices[0].message.content)


Hello! How can I assist you today?


## 데이터 전처리 함수 작성
리뷰 JSON 파일 로드

In [3]:
import json

with open('./res/reviews.json', 'r') as f:
    review_list = json.load(f)
    
review_list[:3]

[{'review': '서울 호캉스를 알아보다가 접근성 좋은 용산역 바로앞에 위치한 나인트리 프리미어 로카우스 호텔 서울 용산!! 정말 기대 이상으로 호텔시설, 애프터눈 티,\r\n수영장, 해피아워, 야경, 조식 이 모든거를 다 누렸는데!\r\n\r\n가격이 20만원 초중반이라니 😍 정말 엄마도 너무 좋아하시고 저도 너무 행복했던 야놀자에 있는 이그제큐티브 라운지 상품 강추합니다👍👍👍\r\n\r\n애프터눈티- 체크인 직후에 라운지에서 티타임❤️\r\n가볍게 샌드위치랑 쿠키가 나왔어요\r\n\r\n수영장- 1,2,3부 중에 원하는 시간 선택해서 이용하는 방식이고 그래서 그런지 다른 수영장 대비 사람이 엄청 많지도 않았어요 카바나 이런거도 무료라 무조건 빨리 입장하셔서 좋은자리 잡는거 추천드려용! 💕\r\n\r\n해피아워- 연어,불고기,치즈 각종음식들이 생각보다 종류도 많고 5성급 해피아워에 밀리지 않는 음식 퀼리티여서 진짜 해피아워 이용하면서 더 반한 로카우스!! 와인, 보드카가 좀 아쉬운거 빼고는 진짜 굿\r\n\r\n야경 정말 완벽 그자체! 제가 3번째 사진에서 찍은것은 해피아워 라운지10층 왼쪽복도 맨끝!!\r\n\r\n조식- 가짓수는 30~40여가지로 엄청 많은것도 적지도 않은 가짓수에요 근데 맛이 다 좋아서 ㅎㅎ\r\n아침부터 폭풍식사 했습니다. 다만 1부, 2부든 무조건 빨리가세오 안그러면 웨이팅 있습니다.\r\n\r\n나인트리 프리미어 로카우스 호텔 서울 용산\r\n서울 호캉스로 가성비도 좋고 기대이상 다 좋아서 만족도 정말 높은 호캉스였어용 또올게용❤️👍',
  'stars': 5,
  'date': '2024.01.24'},
 {'review': '한강뷰가 잘 안보인다는 후기가 있어 걱정했는데 통창으로 너무 잘보였어요!!!! 짱짱!!!',
  'stars': 5,
  'date': '1일 전'},
 {'review': '서울 출장으로 편하게 쉬었습니다', 'stars': 5, 'date': '2일 전'}]

좋은 평점 및 나쁜 평점 정의
- 좋은 평점 = 별점 5개
- 나쁜 평점 = 별점 4개 이하

In [4]:
good_cnt, bad_cnt = 0, 0
for r in review_list:
    if r['stars'] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1

# 좋은 리뷰, 나쁜 리뷰
good_cnt, bad_cnt

(188, 32)

좋은 리뷰 및 나쁜 리뷰로 구분후 저장

In [5]:
reviews_good, reviews_bad = [], []
for r in review_list:
    if r['stars'] == 5:
        reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    else:
        reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

reviews_bad[:3]

['[REVIEW_START]깨끗하고 좋앗어요\n다음에 또가고 싶어요[REVIEW_END]',
 '[REVIEW_START]좋아요~~다음에 재방문할게요[REVIEW_END]',
 '[REVIEW_START]이런 집에서 살고 싶은 청와대 주변과 매우 잘 어울리는 숙소\n\n입구에서 나는 향이 좋고 방 디자인이 미국식 인테리어일거 같다\n미군들도 많이 이용하는거 같다\n\n훌륭하다[REVIEW_END]']

In [6]:
reviews_good_text = '\n'.join(reviews_good)
reviews_bad_text = '\n'.join(reviews_bad)

reviews_bad_text[:100]

'[REVIEW_START]깨끗하고 좋앗어요\n다음에 또가고 싶어요[REVIEW_END]\n[REVIEW_START]좋아요~~다음에 재방문할게요[REVIEW_END]\n[REVIEW_ST'

전처리 함수 작성

In [7]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/reviews.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)
        
    reviews_good, reviews_bad = [], []
    
    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)
    
    for r in review_list:
        review_date_str = r['date']
        # 몇 일전, 몇 시간전 등의 예외 케이스를 대응하기 위한 try~excpet
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            # 최대 일주일간은 현재 날짜로 대체
            review_date = current_date
        
        if review_date < date_boundary:
            continue
        
        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
            
    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)
    
    return reviews_good_text, reviews_bad_text

good, bad = preprocess_reviews()
good[:100]

'[REVIEW_START]한강뷰가 잘 안보인다는 후기가 있어 걱정했는데 통창으로 너무 잘보였어요!!!! 짱짱!!![REVIEW_END]\n[REVIEW_START]서울 출장으로 편하'

## 평가용 함수 작성
평가 기준 설정
- MT-Bench 논문 기반 Pairwise Comparision (=LLM 기반 평가)
    - 비교하는 방식 VS 점수 매기는 방식
    - 점수라는것이 애매할 수 있음(EX. 어느 정도의 요약 품질이 3점인가?)
    - 경험상 점수보다는 비교가 상대적으로 더 정확한 편

평가 스크립트 작성
- MT-Bench 논문 Prompt에서 일부 단어만 수정

In [8]:
def pairwise_eval(reviews, answer_a, answer_b):
    # 원래는 뉴스 비교 프롬프트 지만 조금 수정함
    # 1. x -> korean summaries
    # 2. news -> accommodations
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""

    completion = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role':'user', 'content': eval_prompt}],
        temperature=0
    )
    
    return completion

## Baseline 모델 개발

In [9]:
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

In [10]:
reviews, _ = preprocess_reviews(path='./res/reviews.json')

def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo'):
    prompt = prompt + '\n\n' + reviews
    
    completion = client.chat.completions.create(
         model=model,
         messages=[{'role':'user', 'content': prompt}],
         temperature=temperature
    )
    
    return completion

summarize(reviews, PROMPT_BASELINE).choices[0].message.content

'한강뷰가 잘 안보인다는 우려를 해소하고 통창으로 잘 보였고, 서울 출장으로 편안하게 쉬었다. 방과 야경이 좋았고, 수영장은 작지만 재미있었으며, 주차비가 비싸다는 단점이 있었다. 깨끗하고 친절하며 위치가 좋아서 만족스러웠고, 미국식 인테리어가 세련되고 편안했다.'

In [11]:
summary_real_20241114 = '본 숙소는 용산역과 가까운 뛰어난 접근성을 자랑하며, 주변에 편의시설이 잘 갖추어져 있습니다. 객실은 깨끗하고 넓으며, 한강과 도시 전경을 감상할 수 있는 전망이 좋습니다. 고객 서비스와 직원의 친절함이 우수하며, 조식의 품질도 높습니다. 수영장은 작지만 깔끔하게 관리되고 있으며, 전반적으로 편안한 휴식을 제공합니다. 다만 주차비가 비싼 점은 유의해야 합니다. 전체적으로 매우 만족스러운 숙박 경험을 제공하는 곳입니다.'

In [12]:
print(pairwise_eval(reviews, summarize(reviews, PROMPT_BASELINE).choices[0].message.content, summary_real_20241114).choices[0].message.content)

Assistant A’s summary provides a brief overview of the reviews, mentioning specific aspects such as the view, comfort during a business trip, room quality, and the high parking fee. It also touches on the cleanliness, friendliness, and the American-style interior. However, it lacks depth and does not cover the full range of user reviews.

Assistant B’s summary, on the other hand, offers a more comprehensive overview. It highlights the accommodation's proximity to Yongsan Station, the quality of the rooms, the views, customer service, and breakfast. It also mentions the small but well-maintained swimming pool and the high parking fee. This summary captures a broader range of details from the reviews and provides a more complete picture of the accommodation.

Overall, Assistant B’s response is more detailed and covers more aspects of the user reviews, making it a more helpful and relevant summary.

[[B]]


## 대규모 평가 스크립트
- 원래는 많은 수의 다양한 데이터에 대해 평가를 하지만, 동일한 Prompt에 대해 temperature 높여서 평가 진행

In [13]:
# 10개의 다른 요약을 하여 대조군과 비교하여 얼마나 더 좋고, 좋지 않고, 혹은 무승부인지 확인하기 -> temperature 설정
# 10번은 적다 -> 현재는 6개월치 데이터만 가져왔을 뿐
# 그래도 50번~100번은 사용해야한다 -> gpt-4 비용 급등
eval_count = 10

summaries_baseline = [summarize(reviews, PROMPT_BASELINE, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
summaries_baseline

['1. 한강뷰가 잘 보여서 짱짱했어요!  \n2. 서울 출장으로 편하게 쉬었습니다.  \n3. 방, 야경, 위치 모두 훌륭했는데 주차비가 비싸서 아쉬웠어요.  \n4. 깨끗하고 좋은 호텔, 다음에도 방문하겠다고 합니다.  \n5. 깔끔하고 시설도 좋고, 직원들도 친절해서 만족했다고 합니다.  ',
 '청결한 숙소인데, 전망도 좋고 서비스도 친절하다. 용산역 근처에 위치해 편리하며, 수영장과 조식도 만족스럽고 재방문하고 싶은 숙소다.',
 '- 한강뷰가 통창으로 잘 보였고, 서울 출장으로 편하게 쉴 수 있었던 숙소\n- 방이 좋고 야경이 훌륭하며, 아침에 바라보는 전망도 좋았으나 주차비가 비싸다는 리뷰\n- 깨끗하고 4성급 호텔인데 만족스러웠으며, 위치와 서비스가 좋아서 다음에도 방문할 의사 있음\n- 친절하고 위치가 좋고 시설이 깨끗하며, 미국식인테리어가 세련되면서 편안한 느낌인 호텔\n- 용산역에 가까워서 편리하고, 깨끗하고 직원들이 친절한 호텔로 만족스러운 휴식을 즐겼음',
 '한강뷰를 걱정했지만 통창으로 아주 잘 보였고, 서울 출장으로 편하게 쉴 수 있었으며, 방과 야경이 좋았으며 조식도 만족스럽고, 청결하고 친절하며 위치도 좋았다.입니다.',
 '1. 한강뷰가 통창으로 잘 보여서 만족스러웠음.\n2. 서울 출장으로 편리하게 쉬었음.\n3. 방과 야경 모두 좋았고, 수영장은 작지만 즐거웠음.\n4. 용산역 인근으로 편리하고, 조식이 맛있어서 만족함.\n5. 깨끗하고 친절한 서비스로 인해 전반적으로 만족함.',
 '1. 한강뷰가 통창으로 잘 보였고, 서울 출장으로 편하게 쉬었다고 함.\n2. 방과 야경이 좋았고, 수영장이 작지만 재밌게 놀았다며 재방문 의사 있음.\n3. 깨끗하고 친절하며 위치가 좋으나 주차비가 비싸다고 언급함.\n4. 깨끗하고 만족스러우며 위치와 서비스가 좋다고 하며 재방문 의사 있음.\n5. 일상에서 가장 좋았던 호텔이며, 직원분들이 친절하다고 칭찬함.',
 '다양한 후기들 중에는 통창으로 한강뷰를 즐기는 게 가능하고, 서울 출장으로도

In [14]:
from tqdm import tqdm

def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_cnt, b_cnt, draw_cnt = 0, 0, 0
    for i in tqdm(range(len(answers_a))):
        completion = pairwise_eval(reviews, answers_a[i], answers_b[i])
        verdict_text = completion.choices[0].message.content
        
        if '[[A]]' in verdict_text:
            a_cnt +=1 
        elif '[[B]]' in verdict_text:
            b_cnt += 1
        elif '[[C]]' in verdict_text:
            draw_cnt += 1
        else:
            print('Evalutaion ERROR!')

    return a_cnt, b_cnt, draw_cnt

wins, loses, ties = pairwise_eval_batch(reviews, summaries_baseline, [summary_real_20241114 for _ in range(len(summaries_baseline))])
print(f'Wins: {wins}, Losses: {loses}, Ties: {ties}')

100%|██████████| 10/10 [00:50<00:00,  5.05s/it]

Wins: 1, Losses: 9, Ties: 0


## 모델 고도화 1 - 조건들 명시

In [15]:
# 한글보다는 영어로 작성하면 더 효율적
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0).choices[0].message.content for _ in range(eval_count)]

In [16]:
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20241114 for _ in range(len(summaries))])
print(f'Wins : {wins}, Losses : {losses}, Ties : {ties}')

100%|██████████| 10/10 [00:55<00:00,  5.50s/it]

Wins : 3, Losses : 7, Ties : 0


## 모델 고도화 2 - 입력 데이터의 품질 증가
프롬프트 엔지니어링도 충분히 효과적인 방법론이지만, 데이터 전처리과정을 확실히 한다면 더 효율적이고 효과적이다.

1. 고품질의 리뷰만 필터링 하는 로직 추가 (ex. 좋았어요, 괜찮았어요 등의 단순한 평가는 제외)

In [ ]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/reviews.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)
        
    reviews_good, reviews_bad = [], []
    
    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)
    
    filtered_cnt = 0
    for r in review_list:
        review_date_str = r['date']
        # 몇 일전, 몇 시간전 등의 예외 케이스를 대응하기 위한 try~excpet
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            # 최대 일주일간은 현재 날짜로 대체
            review_date = current_date
        
        if review_date < date_boundary:
            continue
        
        # 단순 길이 제한 
        if len(r['review']) < 30:
            filtered_cnt += 1
            continue
        
        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    
    ## api input token 제한이 발생한다면, 애라와 같이 입력값 제한이 필요(50개)
    # reviews_good = reviews_good[:min(len(reviews_good), 50)]
    # reviews_bad = reviews_bad[:min(len(reviews_bad), 50)]
    
    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)
    
    return reviews_good_text, reviews_bad_text

reviews, _ = preprocess_reviews()

In [ ]:
# 데이터의 길이로만 필터링을 진행했지만 성능 향상 -> eval_count를 더 늘리면 또 모른다.
# 다양한 데이터 전처리 방법을 사용해서 최적화하자
eval_count = 10

summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20241114 for _ in range(len(summaries))])
print(f'Wins : {wins}, Losses : {losses}, Ties : {ties}')

100%|██████████| 10/10 [00:44<00:00,  4.50s/it]

Wins : 4, Losses : 6, Ties : 0


## 모델 고도화 3 - Few-Shot Prompting

1. few-shot을 찾기 위해서 기존 하드코딩된 crawler 로직 변경 (crawler_init -> crawler_update)
2. crawler에서 스크롤 10 -> 20으로 변경(그 이상 설정해도 되지만 시간상으로 20으로 증가)

In [41]:
reviews_1shot, _ = preprocess_reviews('./res/glad_mapo.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0, model='gpt-4o').choices[0].message.content
summary_real_20241115_1shot = '공덕역 인근에 위치하여 교통이 매우 편리하며, 주변에 다양한 편의시설과 맛집이 있어 이용자들에게 좋습니다. 객실은 깨끗하고 넓으며, 침구가 편안하여 숙면을 취하기에 적합합니다. 서비스 또한 친절하다는 평가가 많으며, 셀프 체크인 및 룸서비스 이용이 가능하여 편리함을 제공합니다. 전반적으로 청결하고 쾌적한 분위기로 재방문 의사가 높은 곳입니다. 다만, 주차장 위치나 소음 문제에 대한 개선이 필요하다는 후기도 있습니다.'
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

# prompt = 조건 명시 + prompt_1shot => 결과
# 무조건 조건 명시와 few_shot 한 결과가 더 좋은가? 아래의 결과로 아니다라는 것. 조건 명시만 하는 것이 좋을 수 있고, few_shot만 하는 것이 좋을 수 있고, 둘 다 하는 것이 좋을 수 있다. ---> 많은 테스팅이 필요하다.
eval_count = 10
summaries = [summarize(reviews_1shot, prompt_1shot, temperature=1.0, model='gpt-3.5-turbo').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews_1shot, summaries, [summary_real_20241115_1shot for _ in range(len(summaries))])
print(f'Wins : {wins}, Losses : {losses}, Ties : {ties}')

100%|██████████| 10/10 [00:35<00:00,  3.57s/it]

Wins : 0, Losses : 10, Ties : 0


In [ ]:
# prompt = 조건 명시 => 결과
eval_count = 10
summaries = [summarize(reviews_1shot, prompt, temperature=1.0, model='gpt-3.5-turbo').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews_1shot, summaries, [summary_real_20241115_1shot for _ in range(len(summaries))])
print(f'Wins : {wins}, Losses : {losses}, Ties : {ties}')

100%|██████████| 10/10 [00:41<00:00,  4.18s/it]

Wins : 0, Losses : 10, Ties : 0


In [43]:
# prompt = prompt_1shot => 결과
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews_1shot, prompt_1shot, temperature=1.0, model='gpt-3.5-turbo').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews_1shot, summaries, [summary_real_20241115_1shot for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:55<00:00,  5.53s/it]

Wins: 5, Losses: 5, Ties: 0


### two-shot

In [44]:
reviews_2shot, _ = preprocess_reviews('./res/glad_yeouido.json')
summary_2shot = summarize(reviews_2shot, prompt_1shot, temperature=0.0, model='gpt-4o').choices[0].message.content
summary_real_20241115_2shot = '숙소는 지하철 역과 가까운 편리한 위치에 있으며, 깨끗하고 현대적인 디자인을 갖추고 있습니다. 침구류와 침대의 품질이 우수하여 숙면을 취하기에 적합합니다. 직원들은 친절하고 서비스가 만족스러우며, 조식 또한 호평받고 있습니다. 다만, 일부 객실에서는 전망이 부족하다는 의견이 있으며, 주차 공간이 협소하여 발렛 서비스가 필수적입니다. 전반적으로 가성비가 좋고 재방문 의사가 높은 숙소입니다.'
prompt_2shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.

예시 리뷰들 1:
{reviews_1shot}
예시 요약 결과 1:
{summary_1shot}

예시 리뷰들 2:
{reviews_2shot}
예시 요약 결과 2:
{summary_2shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews_2shot, prompt_2shot, temperature=1.0, model='gpt-3.5-turbo').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews_2shot, summaries, [summary_real_20241115_2shot for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:43<00:00,  4.36s/it]

Wins: 2, Losses: 8, Ties: 0


In [45]:
summaries

['이 숙소는 지하철 역과 가까운 위치로 이동성이 우수한 것으로 평가되고, 깨끗한 침구와 친절한 직원 서비스로 만족스러운 휴식을 제공합니다. 또한, 시설이 현대적이고 깔끔하다는 호평을 받고 있습니다. 다만, 주차장이 협소하고 일부 객실의 전망이 불만족스러워하는 손님들도 있습니다. 하지만 전반적으로 재방문 의사가 높고 만족도가 높은 호텔로 평가받고 있습니다.',
 '이 숙소는 지하철 역과 가까워 이동성이 좋으며, 침구가 편안하여 휴식을 취하기 좋다는 평이 많습니다. 또한, 직원들의 친절한 서비스와 객실의 청결도도 좋다는 평이 있습니다. 다만 뷰가 아쉽거나 주차가 불편하다는 의견도 있습니다. 전반적으로 재방문 의사가 높고, 가격 대비 만족도가 높은 숙소로 평가받고 있습니다.',
 '이 숙소는 지하철 역과 가까워 이동이 편리하며 침구가 편안하고 깔끔하다는 이야기가 많습니다. 서비스도 친절하고 청결한 편이며 재방문 의사가 높은 글래드 호텔로 평가받고 있습니다. 다만 주변에 편의시설이 부족하거나 환기가 조금 부족하다는 의견도 있습니다. 전반적으로 만족스러운 숙박을 제공해주는 것으로 알려져 있습니다.',
 '이 숙소는 교통이 편리하고 객실이 깔끔하며 침구가 푹신하여 휴식을 편안하게 취할 수 있는 호텔로 평가받고 있습니다. 또한, 직원들의 친절한 서비스와 다양한 편의시설이 호감을 얻고 있습니다. 다만, 주차장이 협소한 점이 아쉽다는 의견도 있습니다. 전반적으로 재방문 의사가 높은 호텔로 평가되고 있습니다.',
 '이 숙소는 지하철 역과 가까워 이동성이 좋다는 평가를 받았고, 침구의 퀄리티도 매우 높다는 평을 받았습니다. 전망이 아쉽지만 전체적인 시설은 좋다고 말한 리뷰 내용이 많이 언급되었습니다. 리뷰 중에는 중간 층간소음이 있었지만 전반적으로 깨끗하고 편한 숙박을 즐길 수 있다는 긍정적인 의견도 나왔습니다.특히 스마트 체크인 서비스에 대한 불편함이 언급되었을 뿐, 전체적으로는 만족스러운 투숙평이 많이 있었습니다.',
 '이 숙소는 지하철 역이 가까워 이동성이 좋고, 침구